In [1]:
docs = []
for i in range(102):
    number = i
    filename = "PRSB_{0}.txt".format(number)
    with open ("/Users/tomoatsu/Documents/修士関係/情報システム論実習/情報システム分析論実習/www/Documents/"+filename) as doc:
        doc = doc.read()
        docs.append(doc)

In [2]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context


nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("reuters")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tomoatsu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/tomoatsu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package reuters to
[nltk_data]     /Users/tomoatsu/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to /Users/tomoatsu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
en_stop = nltk.corpus.stopwords.words('english')
def remove_stopwords(word, stopwordset):
  if word in stopwordset:
    return None
  else:
    return word

In [12]:
from nltk.corpus import wordnet as wn #lemmatize関数のためのimport
import re

def preprocessing_text(text):
  def cleaning_text(text):
    pattern1 = '@|%'
    text = re.sub(pattern1, '', text)    
    pattern2 = '\[[0-9 ]*\]'
    text = re.sub(pattern2, '', text)    
    pattern3 = '\([a-z ]*\)'
    text = re.sub(pattern3, '', text)    
    pattern4 = '[0-9]'
    text = re.sub(pattern4, '', text)
    return text
  
  def tokenize_text(text):
    text = re.sub('[.,]', '', text)
    return text.split()

  def lemmatize_word(word):
    # make words lower  example: Python =>python
    word=word.lower()
    
    # lemmatize  example: cooked=>cook
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
      return lemma
    
  text = cleaning_text(text)
  tokens = tokenize_text(text)
  tokens = [lemmatize_word(word) for word in tokens]
  tokens = [remove_stopwords(word, en_stop) for word in tokens]
  tokens = [word for word in tokens if word is not None]
  return tokens

In [13]:
import gensim
from gensim import corpora

In [15]:
pp_docs = [preprocessing_text(text) for text in docs]
dictionary = corpora.Dictionary(pp_docs)
corpus_ = [dictionary.doc2bow(doc) for doc in pp_docs]

In [16]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus_,
                                           num_topics=20,
                                           id2word=dictionary,
                                           alpha=0.1,                             #optional LDAのハイパーパラメータalpha
                                           eta=0.1,                                 #optional LDAのハイパーパラメータbeta
                                           #minimum_probability=0.0    #optional 学習結果に格納するトピック・単語の生起確率の下限
                                          )

In [17]:
topics = ldamodel.print_topics(num_words=15)
for topic in topics:
    print(topic)

(0, '0.005*"use" + 0.004*"suggest" + 0.004*"group" + 0.003*"species" + 0.003*"role" + 0.003*"immobility" + 0.003*"social" + 0.003*"reef" + 0.003*"field" + 0.003*"trait" + 0.003*"change" + 0.003*"diversity" + 0.003*"range" + 0.003*"response" + 0.003*"result"')
(1, '0.009*"species" + 0.007*"predator" + 0.007*"prey" + 0.007*"list" + 0.007*"different" + 0.006*"use" + 0.006*"host" + 0.005*"across" + 0.005*"female" + 0.005*"system" + 0.005*"sperm" + 0.005*"result" + 0.004*"study" + 0.004*"increase" + 0.004*"group"')
(2, '0.010*"species" + 0.007*"population" + 0.004*"across" + 0.004*"prey" + 0.004*"however" + 0.004*"predator" + 0.004*"wa" + 0.003*"variation" + 0.003*"use" + 0.003*"dynamics" + 0.003*"evolution" + 0.003*"fish" + 0.003*"metapopulation" + 0.003*"community" + 0.003*"brain"')
(3, '0.004*"variation" + 0.004*"species" + 0.004*"bird" + 0.004*"change" + 0.003*"forest" + 0.003*"behaviour" + 0.003*"flight" + 0.003*"plague" + 0.003*"population" + 0.003*"level" + 0.003*"worker" + 0.003*"ma

In [18]:
for n,item in enumerate(corpus_[:10]):
    print("document ID "+str(n)+":" ,end="")
    print(ldamodel.get_document_topics(item))

document ID 0:[(13, 0.4762294), (16, 0.5083752)]
document ID 1:[(17, 0.9837566)]
document ID 2:[(8, 0.984294)]
document ID 3:[(5, 0.98515105)]
document ID 4:[(18, 0.98888487)]
document ID 5:[(9, 0.9822376)]
document ID 6:[(2, 0.9842915)]
document ID 7:[(5, 0.21546741), (17, 0.77166575)]
document ID 8:[(10, 0.12699524), (15, 0.85584235)]
document ID 9:[(14, 0.97999704)]


In [19]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus_, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

In [20]:
pyLDAvis.save_html(lda_display,'vis.html')

In [21]:
!ls

Clustering.ipynb                  Pasta_similarity.csv
Clustering_20200716_01.ipynb      Pasta_similarity.xlsx
Clustering_20200716_02.ipynb      README.md
Clustering_EN.ipynb               Similarity.ipynb
Documents                         Similarity_EN.ipynb
LDA_20200716.ipynb                Untitled.ipynb
LDA_EN.ipynb                      nlp_country.csv
LDA_TA.ipynb                      report_20200630.ipynb
NLTK_Kmeans_sample.ipynb          report_20200716_elbow.png
Nations.csv                       report_20200716_silhouette_07.png
Pasta                             vis.html


In [23]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus_,
                                           num_topics=30,
                                           id2word=dictionary,
                                           alpha=0.1,                             #optional LDAのハイパーパラメータalpha
                                           eta=0.1,                                 #optional LDAのハイパーパラメータbeta
                                           #minimum_probability=0.0    #optional 学習結果に格納するトピック・単語の生起確率の下限
                                          )
topics = ldamodel.print_topics(num_words=15)
for n,item in enumerate(corpus_[:10]):
    print("document ID "+str(n)+":" ,end="")
    print(ldamodel.get_document_topics(item))

document ID 0:[(9, 0.9754192)]
document ID 1:[(12, 0.9754183)]
document ID 2:[(1, 0.9762047)]
document ID 3:[(22, 0.9775103)]
document ID 4:[(19, 0.9831355)]
document ID 5:[(0, 0.9731425)]
document ID 6:[(25, 0.9762263)]
document ID 7:[(10, 0.87220937), (21, 0.10791615)]
document ID 8:[(24, 0.97262424)]
document ID 9:[(2, 0.8396974), (19, 0.13112594)]


In [25]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus_, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

In [26]:
pyLDAvis.save_html(lda_display,'vis_topics_30.html')

In [27]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus_,
                                           num_topics=40,
                                           id2word=dictionary,
                                           alpha=0.1,                             #optional LDAのハイパーパラメータalpha
                                           eta=0.1,                                 #optional LDAのハイパーパラメータbeta
                                           #minimum_probability=0.0    #optional 学習結果に格納するトピック・単語の生起確率の下限
                                          )
topics = ldamodel.print_topics(num_words=15)
for n,item in enumerate(corpus_[:10]):
    print("document ID "+str(n)+":" ,end="")
    print(ldamodel.get_document_topics(item))

document ID 0:[(5, 0.9672232)]
document ID 1:[(31, 0.96721894)]
document ID 2:[(35, 0.9682834)]
document ID 3:[(17, 0.530663), (38, 0.44009084)]
document ID 4:[(13, 0.38813126), (23, 0.40943235), (31, 0.18102382)]
document ID 5:[(27, 0.9642131)]
document ID 6:[(9, 0.96828336)]
document ID 7:[(2, 0.7554741), (13, 0.21775243)]
document ID 8:[(36, 0.9635425)]
document ID 9:[(6, 0.9597852)]


In [28]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus_, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

In [29]:
pyLDAvis.save_html(lda_display,'vis_topics_40.html')

In [30]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus_,
                                           num_topics=10,
                                           id2word=dictionary,
                                           alpha=0.1,                             #optional LDAのハイパーパラメータalpha
                                           eta=0.1,                                 #optional LDAのハイパーパラメータbeta
                                           #minimum_probability=0.0    #optional 学習結果に格納するトピック・単語の生起確率の下限
                                          )
topics = ldamodel.print_topics(num_words=15)
for n,item in enumerate(corpus_[:10]):
    print("document ID "+str(n)+":" ,end="")
    print(ldamodel.get_document_topics(item))

document ID 0:[(0, 0.9922379)]
document ID 1:[(1, 0.9404928), (6, 0.052607466)]
document ID 2:[(3, 0.18233974), (8, 0.2916475), (9, 0.52017474)]
document ID 3:[(0, 0.9929098)]
document ID 4:[(7, 0.9947039)]
document ID 5:[(6, 0.99150604)]
document ID 6:[(2, 0.9924969)]
document ID 7:[(2, 0.9935211)]
document ID 8:[(2, 0.9913432)]
document ID 9:[(2, 0.9904216)]


In [31]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus_, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

In [32]:
pyLDAvis.save_html(lda_display,'vis_topics_10.html')